In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

In [5]:
"""
DOCX만 대상으로:
1) 정상 조항(표준계약서/법령) 자동 추출
2) 독소 후보(분쟁문서) 자동 추출 + 자동 태깅(휴리스틱)
3) CSV로 저장(라벨링 가능)

사용 방법:
- 아래 CONFIG만 네 경로로 수정
- python build_clause_dataset_docx_config.py 실행
"""

import re
import csv
import uuid
from dataclasses import dataclass
from pathlib import Path
from typing import List, Dict, Tuple
from docx import Document


@dataclass
class Config:
    NORMAL_DIRS = [
        "data/raw/law/",
        "data/raw/rule/",
    ]
    DISPUTE_DIRS = [
        "data/raw/case/",
    ]
    OUT_CSV = "data/processed/csv/clauses_to_label.csv"
    LIMIT_EACH = 3000
    MIN_LEN = 15
    MAX_LEN = 350


CFG = Config() 
    
# -------------------------
# 1) DOCX 텍스트 추출
# -------------------------
def read_docx(path: Path) -> str:
    doc = Document(str(path))
    parts = []
    for p in doc.paragraphs:
        t = (p.text or "").strip()
        if t:
            parts.append(t)
    return "\n".join(parts)

# -------------------------
# 2) 문장 분리/정제
# -------------------------
# 문장 경계(구분자)를 캡처해서 split: lookbehind 없이 안전
_KO_SENT_SPLIT = re.compile(r"(\.|!|\?|다\.|다\)|함\.|됨\.|한다\.|한다\))\s+")

def split_sentences(text: str, min_len: int, max_len: int):
    lines = []
    for line in (text or "").splitlines():
        line = normalize_text(line)
        if line:
            lines.append(line)

    joined = normalize_text(" ".join(lines))
    if not joined:
        return []

    parts = _KO_SENT_SPLIT.split(joined)

    # parts: [문장조각, 구분자, 문장조각, 구분자, ...]
    sents = []
    buf = ""
    for i, part in enumerate(parts):
        if i % 2 == 0:
            buf = part
        else:
            sent = normalize_text(buf + part)  # 구분자를 다시 붙여서 문장 완성
            if min_len <= len(sent) <= max_len:
                sents.append(sent)

    # 마지막 조각(구분자 없이 끝난 경우)
    if len(parts) % 2 == 1:
        tail = normalize_text(parts[-1])
        if min_len <= len(tail) <= max_len:
            sents.append(tail)

    return sents


# -------------------------
# 3) 독소 후보 자동 태깅(휴리스틱)
# -------------------------
TOXIC_PATTERNS: List[Tuple[str, str, int]] = [
    (r"(정산\s*완료|협의에\s*따른다|상당\s*기간|추후\s*협의)", "반환지연형", 2),
    (r"(보증금.*공제|원상복구.*비용|하자.*비용|추가\s*정산)", "보증금공제확대형", 2),
    (r"(연체|지급기일|지체|납부\s*요청|최고)", "연체가중형", 1),
    (r"(해지\s*통지|계약\s*해지|해제)", "해지압박형", 2),
    (r"(명도|인도\s*절차|퇴거|강제집행)", "명도압박형", 2),
    (r"(임대인.*판단|임대인의\s*재량|임대인이\s*정한|임대인이\s*제시)", "임대인재량형", 2),
    (r"(협조한다|거절하지\s*못한다|서명하여야\s*한다)", "협조강제형", 1),
]

def auto_tag_and_risk(sentence: str) -> Tuple[str, int]:
    tags = []
    risk = 0
    for pat, tag, lvl in TOXIC_PATTERNS:
        if re.search(pat, sentence):
            tags.append(tag)
            risk = max(risk, lvl)
    return (";".join(sorted(set(tags))) if tags else ""), risk

# -------------------------
# 4) 중복 제거
# -------------------------
def dedup_keep_best(rows: List[Dict]) -> List[Dict]:
    best = {}
    for r in rows:
        key = r["text"]
        score = (r.get("risk_level_suggested", 0), -abs(len(key) - 120))
        if key not in best or score > best[key][0]:
            best[key] = (score, r)
    return [v[1] for v in best.values()]

# -------------------------
# 5) 디렉토리에서 문장 추출
# -------------------------
def build_rows_from_dir(dir_path: Path, source_group: str, doc_type: str, cfg: Config) -> List[Dict]:
    rows = []
    for p in sorted(dir_path.rglob("*.docx")):
        text = read_docx(p)
        if not text:
            continue

        sents = split_sentences(text, cfg.MIN_LEN, cfg.MAX_LEN)
        for s in sents:
            if doc_type == "toxic_candidate":
                tags, risk = auto_tag_and_risk(s)
            else:
                tags, risk = "", 0

            rows.append({
                "id": f"{doc_type}_{uuid.uuid4().hex[:12]}",
                "doc_type": doc_type,                 # normal / toxic_candidate
                "source_group": source_group,         # standard_or_law / dispute_docs
                "source_file": str(p),
                "text": s,
                "auto_tags": tags,
                "risk_level_suggested": risk,         # 0 정상, 1 경계, 2 독소후보
                "human_label": "",                    # 네가 나중에 채우는 라벨
                "notes": "",
            })
    return rows

def write_csv(rows: List[Dict], out_csv: Path):
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    cols = ["id","doc_type","source_group","source_file","text","auto_tags","risk_level_suggested","human_label","notes"]
    with out_csv.open("w", newline="", encoding="utf-8-sig") as f:
        w = csv.DictWriter(f, fieldnames=cols)
        w.writeheader()
        for r in rows:
            w.writerow({c: r.get(c, "") for c in cols})

def validate_dirs(cfg: Config):
    for p in cfg.NORMAL_DIRS + cfg.DISPUTE_DIRS:
        pp = Path(p)
        if not pp.exists():
            raise FileNotFoundError(f"폴더가 없습니다: {pp}")

def main(cfg: Config):
    validate_dirs(cfg)

    normal_rows = []
    for nd in cfg.NORMAL_DIRS:
        normal_rows += build_rows_from_dir(Path(nd), "standard_or_law", "normal", cfg)

    toxic_rows = []
    for dd in cfg.DISPUTE_DIRS:
        toxic_rows += build_rows_from_dir(Path(dd), "dispute_docs", "toxic_candidate", cfg)

    normal_rows = dedup_keep_best(normal_rows)[:cfg.LIMIT_EACH]
    toxic_rows  = dedup_keep_best(toxic_rows)[:cfg.LIMIT_EACH]

    all_rows = normal_rows + toxic_rows
    out_csv = Path(cfg.OUT_CSV)
    write_csv(all_rows, out_csv)

    print(f"[OK] normal={len(normal_rows)}, toxic_candidate={len(toxic_rows)}")
    print(f"[SAVED] {out_csv}")

In [6]:
if __name__ == "__main__":
    main(CFG)


NameError: name 'normalize_text' is not defined

In [ ]:
둘 중 하나만 고르라면 **“둘 다”가 제일 성능이 잘 나와.** 다만 역할이 달라.

### A. 법령/시행령/민법/규칙/판례/사례집만 RAG로 찾게 하기

장점

* 근거가 튼튼하고 “정답(법적 기준)”을 잘 가져옴
* 최신/정확한 조문 인용, 요건 정리(2기 연체, 대항력 요건 등)에 강함

한계

* 사용자가 보는 건 **‘계약서 특약 문장’**인데, 법령은 보통 **“그 문장이 위험한 이유”를 직접적으로 말해주지 않음**
* “교묘한 문장(모호한 반환시점, 협조 안 하면 지연 등)”은 **검색이 잘 안 걸리거나** “위법/불공정 판단 포인트”로 연결이 약해질 수 있음

### B. 만든 “독소조항 예시(라벨링 데이터)”를 많이 뽑아 넣기

장점

* 사용자 입력이 계약서 문장일 때, **유사 문장 매칭이 매우 잘 됨**
* “반환지연형/임대인재량형” 같은 **실무형 분류**가 가능해짐

한계

* 예시가 너무 적거나 편향되면 “이런 말투만 독소”로 학습될 수 있음
* 독소 예시만 늘리면 오탐이 늘 수 있어서 **정상 조항/중립 조항도 같이** 필요

---

## 추천 전략 (실제로는 이렇게 많이 함)

### 1) RAG 지식베이스 2트랙

1. **규범 트랙(근거)**:
   `주택임대차보호법/시행령/시행규칙/민법(임대차)/대법원규칙/판례/사례집`
2. **패턴 트랙(탐지)**:
   만든 **독소조항/경계조항/정상조항** 문장 + 라벨(반환지연형 등)

질문이 들어오면

* 패턴 트랙으로 “이 문장 유형이 뭐냐”를 잡고
* 규범 트랙으로 “왜 문제인지/관련 조문·판례 근거”를 붙이는 식

이게 “탐지 + 설명”이 동시에 잘 돼.

---

## 그럼 독소조항 예시는 “몇 개”가 좋냐?

정답은 “가능한 많이”인데, 감을 주면:

* **최소 동작**: 독소 50~100개 + 정상 150~300개
* **쓸만함**: 독소 300~800개 + 정상 1,000~2,000개
* **꽤 탄탄**: 독소 1,500개+ + 정상 5,000개+

여기서 “정상”이 더 많은 게 보통 좋아(현실 계약서에서 정상 문장이 압도적으로 많아서).

또 독소도 한 덩어리로 뽑지 말고 **유형별로 균형**이 중요해:

* 반환지연형
* 보증금공제확대형
* 사후청구형
* 임대인재량형(모호한 ‘협의’, ‘협조’)
* 명도압박형
* 갱신권 제한형(주택임대차는 이쪽도 큼)
* 수선/원상복구 전가형
* 정보비대칭형(선순위/하자/권리관계 고지 회피 등)

---

## 내 추천 결론

* **“법령/판례/사례집만”**으로는 독소조항 ‘탐지’가 약해질 가능성이 큼
* **“독소조항 예시만”**으로는 근거 제시가 약해지고 편향 위험이 큼
* 그래서 **패턴(예시 라벨링) + 근거(법/판례/사례집) 두 축**으로 가는 게 최적